## Анализ данных о бронировании отелей (учебное задание в Pandas)
В настоящей работе анализируются данные о количестве бронирований отелей за двухлетний период (с июля 2015 г. по август 2017 г.).  
Имеющиеся данные отражают информацию об успешных и отмененных заказах, длительности резервирования и других параметрах бронирования для отелей двух различных типов (городские и курортные отели) среди более, чем 100 тыс. бронирований из разных стран.
#### Расшифровка названий переменных датасета:

<font size = 2> Hotel – тип отеля (City Hotel или Resort Hotel)  
Is canceled – бронирование было отменено (1) или нет (0); неотменённое считается успешным  
Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
Arrival full date – полная дата прибытия  
Arrival date year – год прибытия  
Arrival date month – месяц прибытия  
Arrival date week number – номер недели прибытия  
Arrival date day of month – день прибытия  
Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле  
Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле  
Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)  
Adults – число взрослых  
Children – число детей  
Babies – число младенцев  
Meal – выбранный тип питания  
Country – страна происхождения клиента  
Reserved room type – тип зарезервированного номера  
Assigned room type – тип полученного номера (может отличаться от забронированного)  
Customer type – тип бронирования  
Reservation status – значение последнего статуса брони: Canceled – было отменено клиентом; Check-Out – клиент зарегистрировался, но уже покинул отель; No-Show – клиент не зарегистрировался и сообщил администрации отеля причину  
Reservation status date – дата обновления статуса

#### Загрузка и подготовка данных

In [1]:
import pandas as pd

  # Считываем данные

bookings = pd.read_csv('bookings.csv', sep=';')
bookings_head = bookings.head(7)
bookings_head

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [2]:
  # Посмотрим размер данных
bookings.shape

(119390, 21)

In [3]:
  # Посмотрим тип переменных
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [4]:
bookings.dtypes.value_counts()

object     10
int64      10
float64     1
dtype: int64

В исходном датасете названия колонок записаны в неудобном для дальнейшей работы формате. Приведем названия столбцов к нижнему регистру и заменим пробелы на знак нижнего подчёркивания. Например: Is Canceled -> is_canceled

In [5]:
  # Переименуем колонки
    
# (1) bookings.columns = [x.lower().replace(' ', '_') for x in bookings.columns]    через списочное выражение
# (2) bookings.rename(str.lower, axis='columns')  # не работает...
# (3)
bookings.columns = bookings.columns.str.lower().str.replace(' ', '_')
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

#### Исследование данных

Сргуппируем по странам и подсчитаем из каких стран совершалось наибольшее число успешных бронирований.  
Бронирование считается успешным, если в дальнейшем оно не было отменено (переменная is_canceled = 0).  
Из нижеприведенного следует, что страны, входящие в топ-5: Португалия, Великобритания, Франция, Испания и Германия.

In [21]:
bookings.query('is_canceled == 0')  \
.value_counts('country')            
                                     # в данном случае результат подсчета отобразится отсортированным

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
       ...  
SMR        1
MLI        1
SLE        1
MAC        1
PLW        1
Length: 165, dtype: int64

Выясним, на сколько ночей в среднем бронируют отели как типа City Hotel, так и Resort Hotel.

In [7]:
  # Сруппируем датафрейм по типу отеля
  # рассчитаем среднее количество ночей (stays_total_nights) для каждого из двух типов отелей

bookings.groupby('hotel')['stays_total_nights'].mean().round(2)    # округлим

hotel
City Hotel      2.98
Resort Hotel    4.32
Name: stays_total_nights, dtype: float64

Выясним, сколько раз происходил овербукинг.  
В данном примере овербукинг, это изменения типа номера, полученного клиентом, от изначально забронированного.

In [8]:
  # Количество строк в датафрейме
len(bookings)
len(bookings.query('reserved_room_type != assigned_room_type'))

14917

Выясним, в каком месяце полного календарного года (это 2016 г.) было наибольшее количество бронирований.

In [9]:
  # Подсчитаем число бронирований в каждом месяце (arrival_date_month) каждого года (arrival_date_year)

bookings.groupby('arrival_date_year')['arrival_date_month'].value_counts()

arrival_date_year  arrival_date_month
2015               September             5114
                   October               4957
                   August                3889
                   December              2920
                   July                  2776
                   November              2340
2016               October               6203
                   May                   5478
                   April                 5428
                   September             5394
                   June                  5292
                   August                5063
                   March                 4824
                   July                  4572
                   November              4454
                   February              3891
                   December              3860
                   January               2248
2017               May                   6313
                   April                 5661
                   June                  5

Так, в 2016 чаще всего бронировали отель на октябрь, а в 2017 - на май.

Узнаем, на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015, 2016, 2017.

In [10]:
  # В колонке hotel отберем только нужный тип отеля, сгруппируем по году и по месяцу колонку is_canceled,
  # в ней подсчитаем сумму отмен (is_canceled=1)

bookings.query('hotel == "City Hotel"'). \
groupby(['arrival_date_year', 'arrival_date_month'])['is_canceled'] \
.sum().nlargest(7)                                              # nlargest(n, columns) возвращает n макс. значений в указанной колонке

arrival_date_year  arrival_date_month
2017               May                   2217
2016               October               1947
2017               April                 1926
                   June                  1808
2016               June                  1720
                   September             1567
2015               September             1543
Name: is_canceled, dtype: int64

Рассчитаем среднее значение для трех колонок adults, children, babies

In [22]:
bookings[['adults', 'children', 'babies']].mean()


adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

Из трех колонок наибольшее среднее значение у adults. Следовательно, 2 взрослых - наиболее популярный тип размещения.  
Это следует учитывать отелям при планировании организации размещения.

In [12]:
  # Создадим новую колонку total_kids и посчитаем среднее количество всех детей в зависимости от типа отеля

bookings['total_kids'] = bookings['children'] + bookings['babies']
bookings.groupby('hotel')['total_kids'].mean().round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

In [13]:
bookings[['children', 'babies', 'total_kids']].head()

,children,babies,total_kids
0,0.0,0,0.0
1,0.0,0,0.0
2,0.0,0,0.0
3,0.0,0,0.0
4,0.0,0,0.0


#### Расчет коэффициента оттока (Churn_rate)  
Коэффициент оттока - отношение количества отмененных бронирований к общему количеству бронирований.  
Позволяет оценить, сколько клиентов было потеряно.

In [24]:
  # Вычислим относительную частоту значений is_canceled, равных 1, при помощи аргумента normalize
  # Сохраним в новую переменную Churn_rate, выраженную в %

Churn_rate = bookings.value_counts('is_canceled', normalize=True) * 100
Churn_rate

is_canceled
0    62.958372
1    37.041628
dtype: float64

Таким образом, общий процент отмен довольно значителен - 37%!

Посмотрим, связана ли метрика Churn_rate с наличием детей у клиентов.

In [15]:
  # Создадим колонку has_kids, которая принимает значение True, если клиент при бронировании указал
  # хотя бы одного ребенка (total_kids), в противном случае – False

  # 1 вариант

bookings['has_kids'] = bookings['total_kids'] >= 1
bookings[['total_kids', 'has_kids']].head()

,total_kids,has_kids
0,0.0,False
1,0.0,False
2,0.0,False
3,0.0,False
4,0.0,False


In [18]:
  #  Проверим, среди какой группы по наличию детей показатель оттока выше
  # value_counts(normalize=True) - относительная частота

bookings.groupby('has_kids')['is_canceled'].value_counts(normalize=True) * 100

has_kids  is_canceled
False     0              62.778717
          1              37.221283
True      0              65.077154
          1              34.922846
Name: is_canceled, dtype: float64

Рассчитанный показатель оттока для группы, не указавших детей при бронировании, составляет 37,22%.  
В то время как для группы с детьми этот же показатель равен 34,92%.  
Как видно, показатель оттока Churn_rate несколько преобладает у группы, не указавших детей при бронировании.

Однако, в целом две эти характеристики сопоставимы с общим коэффициентом отмен бронирования, равным 37,04%, что говорит о незначительном влиянии факта наличия детей у заказчиков на показатель Churn_rate.